<a href="https://colab.research.google.com/github/nogbazghi/CSC8980/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nahom Ogbazghi
002052292

In [121]:
!pip install conllu
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz


--2021-03-11 04:37:28--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz.3’

review_polarity.tar 100%[===================>]   2.98M  11.0MB/s    in 0.3s    

2021-03-11 04:37:28 (11.0 MB/s) - ‘review_polarity.tar.gz.3’ saved [3127238/3127238]



In [122]:
!tar zxf review_polarity.tar.gz

In [123]:
import os
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using NLTK tokenize all documents, separated by polarity, remove stop words , and list the top 20 most frequent tokens (and their counts) for the positive reviews, and the top 20 most frequent tokens (and their counts). What kind of things do you notice are different between the two sets? (30 points)

I see very little differences. They have overwhelmingly similar similar words, except with different counts. For both, Punctuations were the majority of the largest count tokens as well as words like time, one, movie, like. There were differences: neg had tokens like: !, bad, get; while pos had words like character and story. Additionally some punctuations were more frequent in neg, compared to pos like ? and !.

In [124]:
path = "/content/txt_sentoken/"
posTokens = {}
negTokens = {}
polarity = ["neg/", "pos/"]
for p in polarity:
  for filename in os.listdir(path+p):
    filepath = path+p+filename
    file = open(filepath, 'r')
    content = file.read()
    word_tokens = word_tokenize(content)
    if(p == "neg/"):
      for w in word_tokens:  
        if w not in stop_words:
          if w in negTokens:
            negCountForW = negTokens[w]
            negTokens[w] = negCountForW + 1
          else:
            negTokens[w] = 1
    else:
      for w in word_tokens:  
        if w not in stop_words:
          if w in posTokens:
            posCountForW = posTokens[w]
            posTokens[w] = posCountForW + 1
          else:
            posTokens[w] = 1

sortedNegCount = sorted(negTokens.items(), key=lambda x: x[1], reverse=True)
print("Top 20 Tokens for Negative")
top_neg = sortedNegCount[:20]
for w in top_neg:
  print(w[0], "-", w[1])

sortedPosCount = sorted(posTokens.items(), key=lambda x: x[1], reverse=True)
print("Top 20 Tokens for Positive")
top_pos = sortedPosCount[:20]
for w in top_pos:
  print(w[0], "-", w[1])

Top 20 Tokens for Negative
, - 35269
. - 32162
`` - 9123
's - 8655
) - 5742
( - 5650
film - 4257
n't - 3442
movie - 3174
one - 2637
? - 2201
like - 1832
: - 1540
even - 1381
would - 1185
good - 1126
time - 1111
! - 1056
get - 1039
bad - 1019
Top 20 Tokens for Positive
, - 42448
. - 33714
's - 9473
`` - 8494
) - 6039
( - 6014
film - 5186
one - 2943
n't - 2775
movie - 2497
like - 1713
? - 1570
: - 1502
story - 1231
also - 1200
good - 1190
even - 1175
time - 1171
would - 1079
character - 1067


In [125]:
import pandas as pd

pos_df = []
neg_df = []
sanitized_pos_df = []
sanitized_neg_df = []
posC = 0
negC = 0
for p in polarity:
  for filename in os.listdir(path+p):
    filepath = path+p+filename
    file = open(filepath, 'r')
    content = file.read()
    split_content = content.split()
    if(p == "pos/"):
      d = {"polarity": 1, "text": content}
      df = pd.DataFrame(data=d, index=[posC])
      pos_df.append(df)
      filteredText = ""
      for w in split_content:
        if w not in stop_words:
          filteredText = filteredText + " " + w
      filteredD = {"polarity": 1, "text": filteredText}
      filtered_df = pd.DataFrame(data=d, index=[posC])
      sanitized_pos_df.append(filtered_df)
      posC+=1
    else:
      d = {"polarity": 0, "text": content}
      df = pd.DataFrame(data=d, index=[negC])
      neg_df.append(df)
      filteredText = ""
      for w in split_content:
        if w not in stop_words:
          filteredText = filteredText + " " + w
      filteredD = {"polarity": 1, "text": filteredText}
      filtered_df = pd.DataFrame(data=d, index=[negC])
      sanitized_neg_df.append(filtered_df)
      negC+=1

all_pos = pd.concat(pos_df)
all_neg = pd.concat(neg_df)
print("filtered collection:")
filtered_pos = pd.concat(sanitized_pos_df)
filtered_neg = pd.concat(sanitized_neg_df)

filtered collection:


In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [127]:
vectorizer = TfidfVectorizer()
pos_vectors = vectorizer.fit_transform(all_pos.text)
pos_words_df = pd.DataFrame(pos_vectors.toarray(), columns=vectorizer.get_feature_names())
pos_words_df.head()
pos_X = pos_words_df
pos_y = all_pos.polarity

In [128]:
vectorizer = TfidfVectorizer()
neg_vectors = vectorizer.fit_transform(all_neg.text)
neg_words_df = pd.DataFrame(neg_vectors.toarray(), columns=vectorizer.get_feature_names())
neg_words_df.head()
neg_X = neg_words_df
neg_y = all_neg.polarity

In [129]:
from sklearn.model_selection import train_test_split

# a 50% pos, 70 neg
A_pos_X_train, A_pos_X_test, A_pos_y_train, A_pos_y_test = train_test_split(pos_X, pos_y, train_size=0.5)
A_neg_X_train, A_neg_X_test, A_neg_y_train, A_neg_y_test = train_test_split(neg_X, neg_y, train_size=0.7)
A_X_train = pd.concat([A_pos_X_train, A_neg_X_train])
A_X_train = A_X_train.fillna(0)
A_y_train = pd.concat([A_pos_y_train, A_neg_y_train])
A_y_train = A_y_train.fillna(0)

# b 70% pos, 50 neg
B_pos_X_train, B_pos_X_test, B_pos_y_train, B_pos_y_test = train_test_split(pos_X, pos_y, train_size=0.7)
B_neg_X_train, B_neg_X_test, B_neg_y_train, B_neg_y_test = train_test_split(neg_X, neg_y, train_size=0.5)
B_X_train = pd.concat([B_pos_X_train, B_neg_X_train])
B_X_train = B_X_train.fillna(0)
B_y_train = pd.concat([B_pos_y_train, B_neg_y_train])
B_y_train = B_y_train.fillna(0)

# c 25% pos, 25 neg
C_pos_X_train, C_pos_X_test, C_pos_y_train, C_pos_y_test = train_test_split(pos_X, pos_y, train_size=0.25)
C_neg_X_train, C_neg_X_test, C_neg_y_train, C_neg_y_test = train_test_split(neg_X, neg_y, train_size=0.25)
C_X_train = pd.concat([C_pos_X_train, C_neg_X_train])
C_X_train = C_X_train.fillna(0)
C_y_train = pd.concat([C_pos_y_train, C_neg_y_train])
C_y_train = C_y_train.fillna(0)

In [130]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [131]:
print("Train A NB")
# a
# Create and train a multinomial naive bayes classifier (MultinomialNB)
modelA = MultinomialNB()
modelA.fit(A_X_train, A_y_train)


Train A NB


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [132]:
print("Train B NB")
# b
# Create and train a multinomial naive bayes classifier (MultinomialNB)
modelB = MultinomialNB()
modelB.fit(B_X_train, B_y_train)

Train B NB


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [133]:
print("Train C SVM")
# c
# Create and train a linear support vector classifier (LinearSVC)
modelC = LinearSVC()
modelC.fit(C_X_train, C_y_train)

Train C SVM


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

3. Using the models from question 2, evaluate them on their individual rest of the dataset. This is, for a) 50% positive and 30% negative, for b) 50% negative and 30% positive, and for c) 75% negative and 75% positive. Calculate and show ONLY the following metrics for each model: Accuracy, Precision, Recall, Macro F1-score. (15 points).

In [134]:
A_X_test = pd.concat([A_pos_X_test, A_neg_X_test])
A_X_test = A_X_test.fillna(0)
A_y_test = pd.concat([A_pos_y_test, A_neg_y_test])
A_y_test = A_y_test.fillna(0)

B_X_test = pd.concat([B_pos_X_test, B_neg_X_test])
B_X_test = B_X_test.fillna(0)
B_y_test = pd.concat([B_pos_y_test, B_neg_y_test])
B_y_test = B_y_test.fillna(0)

C_X_test = pd.concat([C_pos_X_test, C_neg_X_test])
C_X_test = C_X_test.fillna(0)
C_y_test = pd.concat([C_pos_y_test, C_neg_y_test])
C_y_test = C_y_test.fillna(0)

In [135]:
def printScores(testData, labels):
  print("Accuracy", sklearn.metrics.accuracy_score(testData, labels)) 
  print("Precision", sklearn.metrics.precision_score(testData, labels))
  print("Recall", sklearn.metrics.recall_score(testData, labels))
  print("F1 Score", sklearn.metrics.f1_score(labels, testData, average='macro'))

In [136]:
import sklearn.metrics

# A
A_true = A_y_test
A_labels = modelA.predict(A_X_test)
print("A Scores")
printScores(A_true, A_labels)

# B
B_true = B_y_test
B_labels = modelB.predict(B_X_test)
print("\nB Scores")
printScores(B_true, B_labels)

# C
C_true = C_y_test
C_labels = modelC.predict(C_X_test)
print("\nC Scores")
printScores(C_true, C_labels)

A Scores
Accuracy 0.375
Precision 0.0
Recall 0.0
F1 Score 0.2727272727272727

B Scores
Accuracy 0.375
Precision 0.375
Recall 1.0
F1 Score 0.2727272727272727


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



C Scores
Accuracy 0.706
Precision 0.710204081632653
Recall 0.696
F1 Score 0.7059705970597059


Using the model performance metrics from question 3, answer the following questions. Please provide logical and intuitive rationale for your answers, simple answers like: because it has the best score, will not be sufficient. (40 points): a) What is the best performing model? b) Why do you think this is the best performing model? c) How does class imbalance play in determining polarity? d) Do you think either more data or a better model is a better approach for this kind of task?




*   The best overall performing model was C. SVM, but A had the highest Precision, and B had the highest Recall.
*   After researching SVM, I believed it was the best performing model because it finds a strong enough difference when the differences of the labeled data are often minimal, and looking at the text provided to us, the difference is minimum.
*   Polarity is impacted because when one label provided more data than the other, there will be more unique words available to one label than the other.
*   More data would only continue down the path of a badly balanced data set. A better model would be ideal if it will take into consideration the class imbalancing and leveling the imbalanced data to avoid mislabeling. 



In [137]:
nltk.download('vader_lexicon')
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [138]:
def polarityScore(docs):
  texts = docs["text"].values
  pol = docs["polarity"].values
  neut = 0
  neg = 0
  pos = 0
  comp = 0
  
  for text in texts:
    score = sia.polarity_scores(text)
    neut += score["neu"]
    neg += score["neg"]
    pos += score["pos"]
    comp += score["compound"]
  neut = neut/len(texts)
  neg = neg/len(texts)
  pos = pos/len(texts)
  comp = comp/len(texts)
  return neut, neg, pos, comp

In [139]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
p_neu, p_neg, p_pos, p_comp = polarityScore(all_pos)
print("Average polarity positive scores:")
print("neutral:",p_neu,"negative:", p_neg, "positive:", p_pos, "compound:", p_comp)

n_neu, n_neg, n_pos, n_comp = polarityScore(all_neg)
print("Average negative positive scores:")
print("neutral:",n_neu, "negative:", n_neg, "positive:", n_pos, "compound:", n_comp)


Average polarity positive scores:
neutral: 0.7596550000000001 negative: 0.09025199999999997 positive: 0.15009199999999998 compound: 0.6479031999999999
Average negative positive scores:
neutral: 0.7631079999999995 negative: 0.11317100000000005 positive: 0.12373599999999989 compound: 0.10536420000000002


Using NLTK and VADER, calculate the sentiment score for all documents in the positive polarity. Calculate the polarity threshold needed (and reasonable) to have the majority of the document labels match. Do the same for the negative class. Provide the threshold needed, the reason why you think this threshold is reasonable, and the accuracy percentage (how many documents are correctly labeled using this threshold). (45 points):

The threshold is the average compound: all compounds divided by the amount of text documents. Additionally, to widen the potential matches the Postive average compound is reduced, as well as the negative average compound score is increased, both by a minimal percentage. For positive, it is increased by 6%, while negative is reduced by 3%. Additional to the compound, I used the negative and positive scores to improve the negative labeling score.
I believe these thresholds are reasonable because for compound there is a steep difference between the positive and negative scores. This allowed me to gain the best return. I would have preferred a better true negative score.

In [140]:
def plusPercentDeviation(num):
  return num + (num * 0.06)

def minusPercentDeviation(num):
  return num - (num * 0.03)

test_p_neu = plusPercentDeviation(p_neu)
test_p_neg = plusPercentDeviation(p_neg)
test_p_pos = minusPercentDeviation(p_pos)
test_p_comp = minusPercentDeviation(p_comp)
test_n_neu = plusPercentDeviation(n_neu)
test_n_neg = minusPercentDeviation(n_neg)
test_n_pos = plusPercentDeviation(n_pos)
test_n_comp = plusPercentDeviation(n_comp)
countP = 0
countN = 0
countFN = 0
countFP = 0

for text in all_pos["text"].values:
  score = sia.polarity_scores(text)
  neut = score["neu"]
  neg = score["neg"]
  pos = score["pos"]
  comp = score["compound"]
  if(comp > test_p_comp):
    countP += 1
  elif(pos > test_p_pos):
    countP += 1
  else:
    countFN += 1

for text in all_neg["text"].values:
  score = sia.polarity_scores(text)
  neut = score["neu"]
  neg = score["neg"]
  pos = score["pos"]
  comp = score["compound"]
  if(comp < test_n_comp):
    countN += 1
  elif(neg > test_n_neg):
    countN += 1
  elif(pos < test_n_pos):
    countN += 1
  else:
    countFP += 1

print("False Positive")
print((countFP/(len(all_neg["text"].values)))*100)
print("False Negative")
print((countFN/(len(all_pos["text"].values)))*100)
print("True Negative")
print((countN/(len(all_neg["text"].values)))*100)
print("True Positives")
print((countP/(len(all_pos["text"].values)))*100)


False Positive
22.3
False Negative
17.599999999999998
True Negative
77.7
True Positives
82.39999999999999


Bonus (40 points): Repeat questions 2,3 and 4 removing all stopwords. Answer the following questions: Did this change the results in any way? Why do you think so?

It did not change the result, I believe that is due to the fact the stop words were irrelevant in terms of value and they were neither unique to either positive or negative texts and thus not impactful to the training.

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [142]:
print("Filtered:")
vectorizer = TfidfVectorizer()
pos_vectors = vectorizer.fit_transform(filtered_pos.text)
pos_words_df = pd.DataFrame(pos_vectors.toarray(), columns=vectorizer.get_feature_names())
pos_words_df.head()
pos_X = pos_words_df
pos_y = filtered_pos.polarity

Filtered:


In [143]:
vectorizer = TfidfVectorizer()
neg_vectors = vectorizer.fit_transform(filtered_neg.text)
neg_words_df = pd.DataFrame(neg_vectors.toarray(), columns=vectorizer.get_feature_names())
neg_words_df.head()
neg_X = neg_words_df
neg_y = filtered_neg.polarity

In [144]:
from sklearn.model_selection import train_test_split

# a 50% pos, 70 neg
A_pos_X_train, A_pos_X_test, A_pos_y_train, A_pos_y_test = train_test_split(pos_X, pos_y, train_size=0.5)
A_neg_X_train, A_neg_X_test, A_neg_y_train, A_neg_y_test = train_test_split(neg_X, neg_y, train_size=0.7)
A_X_train = pd.concat([A_pos_X_train, A_neg_X_train])
A_X_train = A_X_train.fillna(0)
A_y_train = pd.concat([A_pos_y_train, A_neg_y_train])
A_y_train = A_y_train.fillna(0)

# b 70% pos, 50 neg
B_pos_X_train, B_pos_X_test, B_pos_y_train, B_pos_y_test = train_test_split(pos_X, pos_y, train_size=0.7)
B_neg_X_train, B_neg_X_test, B_neg_y_train, B_neg_y_test = train_test_split(neg_X, neg_y, train_size=0.5)
B_X_train = pd.concat([B_pos_X_train, B_neg_X_train])
B_X_train = B_X_train.fillna(0)
B_y_train = pd.concat([B_pos_y_train, B_neg_y_train])
B_y_train = B_y_train.fillna(0)

# c 25% pos, 25 neg
C_pos_X_train, C_pos_X_test, C_pos_y_train, C_pos_y_test = train_test_split(pos_X, pos_y, train_size=0.25)
C_neg_X_train, C_neg_X_test, C_neg_y_train, C_neg_y_test = train_test_split(neg_X, neg_y, train_size=0.25)
C_X_train = pd.concat([C_pos_X_train, C_neg_X_train])
C_X_train = C_X_train.fillna(0)
C_y_train = pd.concat([C_pos_y_train, C_neg_y_train])
C_y_train = C_y_train.fillna(0)

In [145]:
print("Train A NB")
# a
# Create and train a multinomial naive bayes classifier (MultinomialNB)
filtmodelA = MultinomialNB()
filtmodelA.fit(A_X_train, A_y_train)

Train A NB


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [146]:
print("Train B NB")
# b
# Create and train a multinomial naive bayes classifier (MultinomialNB)
filtmodelB = MultinomialNB()
filtmodelB.fit(B_X_train, B_y_train)

Train B NB


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [147]:
print("Train C SVM")
# c
# Create and train a linear support vector classifier (LinearSVC)
filtmodelC = LinearSVC()
filtmodelC.fit(C_X_train, C_y_train)

Train C SVM


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [148]:
A_X_test = pd.concat([A_pos_X_test, A_neg_X_test])
A_X_test = A_X_test.fillna(0)
A_y_test = pd.concat([A_pos_y_test, A_neg_y_test])
A_y_test = A_y_test.fillna(0)

B_X_test = pd.concat([B_pos_X_test, B_neg_X_test])
B_X_test = B_X_test.fillna(0)
B_y_test = pd.concat([B_pos_y_test, B_neg_y_test])
B_y_test = B_y_test.fillna(0)

C_X_test = pd.concat([C_pos_X_test, C_neg_X_test])
C_X_test = C_X_test.fillna(0)
C_y_test = pd.concat([C_pos_y_test, C_neg_y_test])
C_y_test = C_y_test.fillna(0)

In [149]:
def printScores(testData, labels):
  print("Accuracy", sklearn.metrics.accuracy_score(testData, labels)) 
  print("Precision", sklearn.metrics.precision_score(testData, labels))
  print("Recall", sklearn.metrics.recall_score(testData, labels))
  print("F1 Score", sklearn.metrics.f1_score(labels, testData, average='macro'))

In [150]:
import sklearn.metrics

# A
A_true = A_y_test
A_labels = filtmodelA.predict(A_X_test)
print("A Scores")
printScores(A_true, A_labels)

# B
B_true = B_y_test
B_labels = filtmodelB.predict(B_X_test)
print("\nB Scores")
printScores(B_true, B_labels)

# C
C_true = C_y_test
C_labels = filtmodelC.predict(C_X_test)
print("\nC Scores")
printScores(C_true, C_labels)

A Scores
Accuracy 0.375
Precision 0.0
Recall 0.0
F1 Score 0.2727272727272727

B Scores
Accuracy 0.375
Precision 0.375
Recall 1.0
F1 Score 0.2727272727272727


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



C Scores
Accuracy 0.7213333333333334
Precision 0.7106598984771574
Recall 0.7466666666666667
F1 Score 0.7211543764086819


In [151]:
def polarityScore(docs):
  texts = docs["text"].values
  pol = docs["polarity"].values
  neut = 0
  neg = 0
  pos = 0
  comp = 0
  
  for text in texts:
    score = sia.polarity_scores(text)
    neut += score["neu"]
    neg += score["neg"]
    pos += score["pos"]
    comp += score["compound"]
  neut = neut/len(texts)
  neg = neg/len(texts)
  pos = pos/len(texts)
  comp = comp/len(texts)
  return neut, neg, pos, comp

In [152]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
p_neu, p_neg, p_pos, p_comp = polarityScore(filtered_pos)
print("Filtered")
print("Average polarity positive scores:")
print("neutral:",p_neu,"negative:", p_neg, "positive:", p_pos, "compound:", p_comp)

n_neu, n_neg, n_pos, n_comp = polarityScore(filtered_neg)
print("Average negative positive scores:")
print("neutral:",n_neu, "negative:", n_neg, "positive:", n_pos, "compound:", n_comp)


Filtered
Average polarity positive scores:
neutral: 0.7596550000000001 negative: 0.09025199999999997 positive: 0.15009199999999998 compound: 0.6479031999999999
Average negative positive scores:
neutral: 0.7631079999999995 negative: 0.11317100000000005 positive: 0.12373599999999989 compound: 0.10536420000000002
